In [4]:
!pip install pandas sqlalchemy pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.3 MB/s eta 0:00:00


In [5]:
from sqlalchemy import create_engine
import pandas as pd

In [6]:
ciudad = pd.read_csv('/content/ciudad.csv')
estudiantes = pd.read_csv('/content/estudiantes.csv')

In [7]:
engine = create_engine('sqlite:///:memory:')

In [8]:
ciudad.to_sql('ciudad', engine, index=False)
estudiantes.to_sql('estudiantes', engine, index=False)

1000

In [12]:
estudiantes.head()

,ID,NOMBREAPELLIDO,CODIGOCIUDAD,FECHAINGRESO,TELEFONO,CORREO,CARRERA
0,1,EDELKA,1,3/5/2019,2579148,edelka@hotmail.com,Ingeniería de sistemas
1,2,LUAN,5,10/6/2019,3925183,luan@hotmail.com,Ingenieria química
2,3,SIOMER,2,3/7/2020,2787650,siomer@hotmail.c,Ingeniería de sistemas
3,4,ZAHORY,2,12/8/2019,3868566,zahory@hotmail.com,Ingeniería de sistemas
4,5,HICTADOR,1,3/9/2019,3047140,hictador@hotmail.com,Ingeniería de sistemas


#Primera pregunta:

Cantidad de estudiantes por ciudad. Adicionalmente, ordenar estas ciudades por cantidad de estudiantes en ella de mayor a menor.
Nota: Tener en cuenta que la tabla "Ciudad" y la tabla "estudiante" tienen la llave codigociudad

In [9]:
pregunta1 = '''
SELECT c.NOMBRECIUDAD AS Ciudad, COUNT(e.Id) AS Cantidad_Estudiantes
FROM Ciudad c
LEFT JOIN Estudiantes e ON c.CODIGOCIUDAD = e.CODIGOCIUDAD
GROUP BY c.NOMBRECIUDAD
ORDER BY Cantidad_Estudiantes DESC;
'''
resultado1 = pd.read_sql(pregunta1, engine)
resultado1.head()

,Ciudad,Cantidad_Estudiantes
0,Barranquilla,188
1,Bogotá,148
2,Medellín,138
3,Villavicencio,69
4,Santa Marta,68


#Segunda pregunta

Agrupar la cantidad de estudiantes por año de ingreso y carrera.

In [18]:
pregunta2 = '''
SELECT substr(FECHAINGRESO, -4) AS Año_Ingreso, CARRERA, COUNT(Id) AS Cantidad_Estudiantes
FROM Estudiantes
GROUP BY Año_Ingreso, CARRERA
HAVING Año_Ingreso IS NOT NULL;
'''
resultado2 = pd.read_sql(pregunta2, engine)
resultado2.head()

,Año_Ingreso,CARRERA,Cantidad_Estudiantes
0,2016,Administración de empresas,6
1,2016,Ciencia política,10
2,2016,Estadística,9
3,2016,Geología,12
4,2016,Ingenieria electrónica,8


#Tercera Pregunta:

Mostrar cuantos meses han transcurrido desde la fecha de ingreso a hoy (1/10/2023) por cada estudiate.

In [22]:
pregunta3 = '''
SELECT Id,
       (CAST(SUBSTR('2023-10-01', 1, 4) AS INTEGER) - CAST(SUBSTR(FECHAINGRESO, -4) AS INTEGER)) * 12 +
       (CAST(SUBSTR('2023-10-01', 6, 2) AS INTEGER) - CAST(SUBSTR(FECHAINGRESO, 5, 2) AS INTEGER)) AS Meses_Transcurridos
FROM Estudiantes;
'''
resultado3 = pd.read_sql(pregunta3, engine)
resultado3.head()

,ID,Meses_Transcurridos
0,1,38.0
1,2,58.0
2,3,26.0
3,4,58.0
4,5,38.0


#Cuarta Pregunta:

Seleccionar las ciudades en las cuales no tienen estudiantes.

In [23]:
pregunta4 = '''
SELECT c.NOMBRECIUDAD AS Ciudad
FROM Ciudad c
LEFT JOIN Estudiantes e ON c.CODIGOCIUDAD = e.CODIGOCIUDAD
WHERE e.Id IS NULL;
'''
resultado4 = pd.read_sql(pregunta4, engine)
resultado4.head()

,Ciudad
0,Cali
1,Neiva
2,Armenia
3,Valledupar


#Quinta Pregunta:

Seleccionar los estudiantes que no tienen Telefono.

In [24]:
pregunta5 = '''
SELECT *
FROM Estudiantes
WHERE TELEFONO IS NULL OR TELEFONO = '';
'''
resultado5 = pd.read_sql(pregunta5, engine)
resultado5.head()

,ID,NOMBREAPELLIDO,CODIGOCIUDAD,FECHAINGRESO,TELEFONO,CORREO,CARRERA
0,233,ERCIRIA,4,7/26/2018,None,erciria@gmail.co,Ingeniería Industrial
1,366,MARIAIS,10,6/3/2021,None,mariais@gmail.co,Ciencia política
2,527,LAYNA,7,5/10/2023,None,layna@gmail.co,Geología
3,591,ALENAY,7,6/26/2019,None,alenay@gmail.com,Ingeniería Industrial
4,633,GIME,1,6/5/2021,None,gime@outlook.co,Ciencia política


#Sexta Pregunta:

 Identificar la cantidad de estudiantes por carrera en cada ciudad.

In [25]:
pregunta6 = '''
SELECT c.NOMBRECIUDAD AS Ciudad, e.CARRERA, COUNT(e.Id) AS Cantidad_Estudiantes
FROM Ciudad c
LEFT JOIN Estudiantes e ON c.CODIGOCIUDAD = e.CODIGOCIUDAD
GROUP BY c.NOMBRECIUDAD, e.CARRERA;
'''
resultado6 = pd.read_sql(pregunta6, engine)
resultado6.head()

,Ciudad,CARRERA,Cantidad_Estudiantes
0,Armenia,None,0
1,Barranquilla,Administración de empresas,16
2,Barranquilla,Ciencia política,17
3,Barranquilla,Estadística,24
4,Barranquilla,Geología,21


#Séptima Pregunta:

Valide que el campo "correo" sea consistente  de lo contrario se debe marcar nulo. (validar al menos que contenga el caracter '@')

In [27]:
pregunta7 = '''
SELECT Id,
       CASE
           WHEN instr(CORREO, '@') > 0 THEN 'Consistente'
           ELSE 'Nulo'
       END AS Validacion_Correo
FROM Estudiantes;
'''
resultado7 = pd.read_sql(pregunta7, engine)
resultado7.head()

,ID,Validacion_Correo
0,1,Consistente
1,2,Consistente
2,3,Consistente
3,4,Consistente
4,5,Consistente
